# Module

In [1]:
import glob
import pandas as pd
from llama_index.core import Document
from tqdm import tqdm
import os
import sys
import dotenv
dotenv.load_dotenv()
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

from llama_index.core import SimpleDirectoryReader
from llama_index.core.schema import MetadataMode
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.vector_stores.milvus import MilvusVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.embeddings.openai import OpenAIEmbedding

from pymilvus import Collection, connections
import time

import openai
from llama_index.llms.openai import OpenAI

# Load

In [2]:
parquet_file = "/home/livin/rag_pipeline/wikipedia_rag/data/wikipedia/wiki_ko.parquet"
df = pd.read_parquet(parquet_file)

In [5]:
df

,id,url,title,text
0,5,https://ko.wikipedia.org/wiki/%EC%A7%80%EB%AF%...,지미 카터,"제임스 얼 카터 주니어(, 1924년 10월 1일~)는 민주당 출신 미국의 제39대..."
1,9,https://ko.wikipedia.org/wiki/%EC%88%98%ED%95%99,수학,"수학(數學, , 줄여서 math)은 수, 양, 구조, 공간, 변화 등의 개념을 다루..."
2,10,https://ko.wikipedia.org/wiki/%EC%88%98%ED%95%...,수학 상수,"수학에서 상수란 그 값이 변하지 않는 불변량으로, 변수의 반대말이다. 물리 상수와는..."
3,19,https://ko.wikipedia.org/wiki/%EB%AC%B8%ED%95%99,문학,"문학(文學, )은 언어를 예술적 표현의 제재로 삼아 새로운 의미를 창출하여, 인간과..."
4,20,https://ko.wikipedia.org/wiki/%EB%82%98%EB%9D%...,나라 목록,이 목록에 실린 국가 기준은 1933년 몬테비데오 협약 1장을 참고로 하였다. 협정...
...,...,...,...,...
647892,3708806,https://ko.wikipedia.org/wiki/%ED%98%B8%ED%8A%...,호튼 푸트,'그는 완전히 미쳤다' 라는 말이 가장 잘 어울리는 남자이다.\n\n그니깐 내말은 ...
647893,3708807,https://ko.wikipedia.org/wiki/2023%EB%85%84%20...,2023년 대한민국 FA컵 결승전,대한민국 FA컵 2023 결승전은 2023년 대한민국 FA컵의 우승팀을 가리기 위한...
647894,3708809,https://ko.wikipedia.org/wiki/%EC%8A%A4%ED%84%...,스턴트 맨 (1980년 영화),넘겨주기 스턴트맨 (1980년 영화)
647895,3708814,https://ko.wikipedia.org/wiki/%EC%B9%A0%EC%9B%...,칠원윤씨 윤일상 계후입안,칠원윤씨 가선대부 증병조판서 행삼도통어사 오위도총부 부총관 윤시달 가문의 가계 계...


In [ ]:
output_dir = "../data/wiki_ko_txt/"

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

for index, row in tqdm(df.iterrows()):
    title = row['title']
    title = title.replace('/', '_')
    title += f"_{index}"
    text = row['text']
    file_path = os.path.join(output_dir, f"{title}.txt")
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(text)

647897it [00:54, 11789.60it/s]


In [2]:
a = glob.glob("/home/livin/rag_pipeline/wikipedia_rag/data/wiki_ko_txt/*")

In [3]:
len(a)

647549

In [4]:
a

['/home/livin/rag_pipeline/wikipedia_rag/data/wiki_ko_txt/KITTE.txt',
 '/home/livin/rag_pipeline/wikipedia_rag/data/wiki_ko_txt/헤클러운트코흐 AG36.txt',
 '/home/livin/rag_pipeline/wikipedia_rag/data/wiki_ko_txt/1998년 아시안 게임 펜싱.txt',
 '/home/livin/rag_pipeline/wikipedia_rag/data/wiki_ko_txt/낯선 도시에서의 하루.txt',
 '/home/livin/rag_pipeline/wikipedia_rag/data/wiki_ko_txt/화순 우봉리 들소리.txt',
 '/home/livin/rag_pipeline/wikipedia_rag/data/wiki_ko_txt/영가.txt',
 '/home/livin/rag_pipeline/wikipedia_rag/data/wiki_ko_txt/마이클 임피리올리.txt',
 '/home/livin/rag_pipeline/wikipedia_rag/data/wiki_ko_txt/MUSIC FOR THE PEOPLE.txt',
 '/home/livin/rag_pipeline/wikipedia_rag/data/wiki_ko_txt/포라노.txt',
 '/home/livin/rag_pipeline/wikipedia_rag/data/wiki_ko_txt/하남종합운동장 보조경기장.txt',
 '/home/livin/rag_pipeline/wikipedia_rag/data/wiki_ko_txt/지리좌표 거리.txt',
 '/home/livin/rag_pipeline/wikipedia_rag/data/wiki_ko_txt/작은피그미날다람쥐.txt',
 '/home/livin/rag_pipeline/wikipedia_rag/data/wiki_ko_txt/페이돈.txt',
 '/home/livin/rag_pipeline/wikipedia